In [1]:
import cv2
import os
from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from keras.utils import normalize
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.models import load_model


image_directory = 'brain_tumor_dataset/'
no_tumour = os.listdir(image_directory + 'no/')
yes_tumour = os.listdir(image_directory + 'yes/')

dataset = []
label = []
INPUT_SIZE = 64

#image processing 
for i, image_name in enumerate(no_tumour):
    if image_name.split('.')[1] == 'jpg':
        image = cv2.imread(image_directory + 'no/' + image_name)
        image = Image.fromarray(image, 'RGB')
        image = image.resize((INPUT_SIZE, INPUT_SIZE))
        dataset.append(np.array(image))
        label.append(0)


for i, image_name in enumerate(yes_tumour):
    if image_name.split('.')[1] == 'jpg':
        image = cv2.imread(image_directory + 'yes/' + image_name)
        image = Image.fromarray(image, 'RGB')
        image = image.resize((INPUT_SIZE, INPUT_SIZE))
        dataset.append(np.array(image))
        label.append(1)

dataset = np.array(dataset)
label = np.array(label)

# training and testing sets
x_train, x_test, y_train, y_test = train_test_split(dataset, label, test_size=0.2, random_state=0)


x_train = normalize(x_train, axis=1)
x_test = normalize(x_test, axis=1)

#  model
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(INPUT_SIZE, INPUT_SIZE, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))


model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=16, verbose=1, epochs=10, validation_data=(x_test, y_test), shuffle=True)

model.save('BrainTumour10Epochs.h5')

model = load_model('BrainTumour10Epochs.h5')

# processing test image 
image = cv2.imread('brain_tumor_dataset/yes/Y10.jpg')
img = Image.fromarray(image)
img = img.resize((64, 64))
img = np.array(img)

input_img = np.expand_dims(img, axis=0)

predictions = model.predict(input_img)
result = (predictions > 0.5).astype("int32")
print(result)



C:\Users\hp\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 84ms/step - accuracy: 0.5672 - loss: 0.7064 - val_accuracy: 0.4857 - val_loss: 0.6807
Epoch 2/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.6084 - loss: 0.6699 - val_accuracy: 0.6286 - val_loss: 0.6652
Epoch 3/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.5485 - loss: 0.6888 - val_accuracy: 0.7714 - val_loss: 0.6423
Epoch 4/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.6311 - loss: 0.6590 - val_accuracy: 0.8000 - val_loss: 0.5923
Epoch 5/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.6849 - loss: 0.6056 - val_accuracy: 0.8286 - val_loss: 0.5360
Epoch 6/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.6536 - loss: 0.5833 - val_accuracy: 0.8000 - val_loss: 0.5035
Epoch 7/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.7200 - loss: 0.6172 - val_accuracy: 0.6286 - val_loss: 0.6124
Epoch 8/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.7116 - loss: 0.6250 - val_accuracy: 0.8000 - val_loss: 0.5834


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step
[[1]]


In [ ]:
from flask import Flask, request, render_template_string, redirect, url_for
from werkzeug.utils import secure_filename
import os
import numpy as np
from PIL import Image
import cv2
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score

from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)  

# folder for uploaded files 
UPLOAD_FOLDER = 'uploads'
if not os.path.exists(UPLOAD_FOLDER):
    os.makedirs(UPLOAD_FOLDER)

app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

model = load_model('BrainTumour10Epochs.h5')


INPUT_SIZE = 64

# test data loaded 
def load_test_data():
    test_images = []
    test_labels = []

    for i, image_name in enumerate(os.listdir('brain_tumor_dataset/yes/')):
        if image_name.split('.')[1] == 'jpg':
            image = cv2.imread('brain_tumor_dataset/yes/'+image_name)
            image = Image.fromarray(image, 'RGB')
            image = image.resize((INPUT_SIZE, INPUT_SIZE))
            test_images.append(np.array(image))
            test_labels.append(1)

    for i, image_name in enumerate(os.listdir('brain_tumor_dataset/no/')):
        if image_name.split('.')[1] == 'jpg':
            image = cv2.imread('brain_tumor_dataset/no/'+image_name)
            image = Image.fromarray(image, 'RGB')
            image = image.resize((INPUT_SIZE, INPUT_SIZE))
            test_images.append(np.array(image))
            test_labels.append(0)

    test_images = np.array(test_images)
    test_labels = np.array(test_labels)

    return test_images, test_labels


def calculate_accuracy(y_true, y_pred):
    return accuracy_score(y_true, y_pred)

@app.route('/', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        if 'file' not in request.files:
            return redirect(request.url)
        file = request.files['file']
        if file.filename == '':
            return redirect(request.url)
        if file:
            filename = secure_filename(file.filename)
            file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
            file.save(file_path)
            result, accuracy = predict_tumor(file_path)
            return render_template_string(result_template, prediction=result, accuracy=accuracy)
    return render_template_string(upload_template)

#processing image and prediction 
def predict_tumor(img_path):
    image = cv2.imread(img_path)
    image = Image.fromarray(image, 'RGB')
    image = image.resize((INPUT_SIZE, INPUT_SIZE))
    img = np.array(image)
    input_img = np.expand_dims(img, axis=0)
    input_img = input_img / 255.0
    predictions = model.predict(input_img)
    result = 'Tumor' if predictions[0][0] > 0.5 else 'No Tumor'
    
    test_images, test_labels = load_test_data()
    
    y_pred = []
    for image in test_images:
        prediction = model.predict(np.expand_dims(image, axis=0))
        y_pred.append(int(np.round(prediction[0, 0])))
    
    accuracy = calculate_accuracy(test_labels, y_pred)
    
    return result, accuracy

upload_template = """
<!doctype html>
<html lang="en">
  <head>
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">
    <link href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css" rel="stylesheet">
    <title>Brain Tumor Detection</title>
    <style>
      body {
        background-color: #f8f9fa;
      }
      .container {
        margin-top: 50px;
        max-width: 500px;
      }
      .card {
        padding: 20px;
        border-radius: 10px;
        text-align: center;
      }
      .logo {
        max-width: 100px; 
        margin: 0 auto 20px auto;
        display: block;
      }
    </style>
  </head>
  <body>
    <div class="container">
      <div class="card">
        <img src="{{ url_for('static', filename='logo.png') }}" alt="Logo" class="logo">
        <h1 class="text-center">Brain Tumor Detection</h1>
        <form method="post" enctype="multipart/form-data">
          <div class="form-group">
            <label for="file">Upload an MRI Image</label>
            <input type="file" class="form-control-file" name="file" accept="image/*" required>
          </div>
          <button type="submit" class="btn btn-primary btn-block">Upload</button>
        </form>
      </div>
    </div>
  </body>
</html>

"""

result_template = """
<!doctype html>
<html lang="en">
  <head>
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">
    <link href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css" rel="stylesheet">
    <title>Prediction Result</title>
    <style>
      body {
        background-color: #f8f9fa;
      }
      .container {
        margin-top: 50px;
        max-width: 500px;
      }
      .card {
        padding: 20px;
        border-radius: 10px;
        text-align: center;
      }
      .logo {
        max-width: 100px; 
        margin: 0 auto 20px auto;
        display: block;
      }
      .result {
        margin-top: 20px;
      }
    </style>
  </head>
  <body>
    <div class="container">
      <div class="card">
        <img src="{{ url_for('static', filename='logo.png') }}" alt="Logo" class="logo">
        <h1 class="text-center">Prediction Result</h1>
        <p class="text-center result">Prediction: {{ prediction }}</p>
        <p class="text-center result">Accuracy: {{ accuracy }}</p>
        <a href="{{ url_for('upload_file') }}" class="btn btn-primary btn-block">Upload Another Image</a>
      </div>
    </div>
  </body>
</html>

"""

if __name__ == "__main__":
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [05/Oct/2024 00:57:55] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [05/Oct/2024 00:57:55] "GET /static/logo.png HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [05/Oct/2024 00:57:55] "GET /favicon.ico HTTP/1.1" 404 -
Exception in thread Thread-5:
Traceback (most recent call last):
  File "C:\Users\hp\anaconda3\Lib\site-packages\urllib3\connection.py", line 203, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\hp\anaconda3\Lib\site-packages\urllib3\util\connection.py", line 85, in create_connection
    raise err
  File "C:\Users\hp\anaconda3\Lib\site-packages\urllib3\util\connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [WinError 10061] No connection co

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 222ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 211ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━

INFO:werkzeug:127.0.0.1 - - [05/Oct/2024 00:58:24] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [05/Oct/2024 00:58:24] "GET /static/logo.png HTTP/1.1" 404 -
